In [81]:
import scsims 
import pandas as pd 
import numpy as np
import os
import anndata as an
from scsims import SIMS 
import scanpy as sc
import anndata as an
from torch.utils.data import DataLoader
from scsims.testing import TestAnndatasetMatrix
import torch
from tqdm import tqdm
from scsims import clean_sample 
import numpy as np
import pandas as pd 
from scsims import SIMSClassifier
from scsims.data import AnnDatasetMatrix
import plotly.express as px 
from scsims import DataModule
import os 
from pytorch_lightning.loggers import WandbLogger
import pytorch_lightning as pl 

from bigcsv import to_h5ad

class UploadCallback(pl.callbacks.Callback):
    def __init__(
        self, 
        path: str, 
        desc: str, 
        upload_path='model_checkpoints',
        epochs: int=1,
    ) -> None:
        super().__init__()
        self.path = path 
        self.desc = desc
        self.upload_path = upload_path
        self.epochs = epochs 

    def on_train_epoch_end(self, trainer, pl_module):
        epoch = trainer.current_epoch

        if epoch % self.epochs == 0 and epoch > 0: # Save every ten epochs
            checkpoint = f'checkpoint-{epoch}-desc-{self.desc}.ckpt'
            trainer.save_checkpoint(os.path.join(self.path, checkpoint))
            print(f'Saving checkpoint at epoch {epoch}')

wandb: Network error (ConnectionError), entering retry loop.


In [9]:
df = pd.read_csv('../data/SCP282/expression/expression_11a.6mon.txt', nrows=2, sep='\t')

In [10]:
df

,GENE,11a.6mon_X1_AAACCTGAGAATCTCC,11a.6mon_X1_AAACCTGAGACAGACC,11a.6mon_X1_AAACCTGAGATGTAAC,11a.6mon_X1_AAACCTGAGGAGTTGC,11a.6mon_X1_AAACCTGCAATGGTCT,11a.6mon_X1_AAACCTGCAGATGGCA,11a.6mon_X1_AAACCTGCAGCTGTGC,11a.6mon_X1_AAACCTGCAGGCTCAC,11a.6mon_X1_AAACCTGCATCACGTA,...,11a.6mon_X3_TTTGTCAAGTTGCAGG,11a.6mon_X3_TTTGTCACAACTGGCC,11a.6mon_X3_TTTGTCACAATGCCAT,11a.6mon_X3_TTTGTCACATAAAGGT,11a.6mon_X3_TTTGTCAGTAAGTGTA,11a.6mon_X3_TTTGTCAGTACATCCA,11a.6mon_X3_TTTGTCAGTAGGAGTC,11a.6mon_X3_TTTGTCAGTGTAACGG,11a.6mon_X3_TTTGTCAGTTTGACAC,11a.6mon_X3_TTTGTCATCAACACCA
0,RP11-34P13.3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,FAM138A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# import concurrent.futures

# infiles = [f for f in os.listdir('../data/SCP282/expression/') if f.startswith('expression')]
# outfiles = ['_'.join(f.split('.')[:-1]) + '.h5ad' for f in files]

# with concurrent.futures.ProcessPoolExecutor(max_workers=8) as executor:
#     for infile, outfile in zip(infiles, outfiles):
#         executor.submit(
#             to_h5ad,
#             file=f'../data/SCP282/expression/{infile}',
#             outfile=f'../data/SCP282/{outfile}',
#             sep='\t',
#             sparsify=True,
#             chunksize=400,
#             index_col='GENE',
#             quiet=False,
#         )


In [15]:
h5s = [f for f in os.listdir('../data/SCP282/') if f.endswith('h5ad') and f.startswith('expression')]

h5s

['expression_PGP1_6mon_Batch2.h5ad',
 'expression_PGP1_3mon.h5ad',
 'expression_PGP1_3mon_batch2.h5ad',
 'expression_11a_6mon.h5ad',
 'expression_HUES66_3mon.h5ad',
 'expression_PGP1_6mon.h5ad']

In [16]:
# for file in h5s:
#     data = an.read_h5ad(f'../data/SCP282/{file}')
#     data = data.T
#     data.write_h5ad(f'../data/SCP282/for_inference/transposed_{file}')

In [16]:
h5s = [f for f in os.listdir('../data/SCP282/for_inference') if f.endswith('h5ad')]

len(h5s)

6

In [18]:
l = an.read_h5ad(f'../data/SCP282/for_inference/{h5s[0]}')

for file in h5s:
    data = an.read_h5ad(f'../data/SCP282/for_inference/{file}')
    print(all(data.var['index'] == l.var['index']))

True
True
True
True
True
True


In [18]:
l = an.read_h5ad(f'../data/SCP282/for_inference/{h5s[0]}')
l.obs

""
PGP1.6mon.Batch2_X1_AAACCTGAGATCTGAA
PGP1.6mon.Batch2_X1_AAACCTGAGCCCAATT
PGP1.6mon.Batch2_X1_AAACCTGAGGATGCGT
PGP1.6mon.Batch2_X1_AAACCTGCACTTCTGC
PGP1.6mon.Batch2_X1_AAACCTGGTATATCCG
...
PGP1.6mon.Batch2_X3_TTTGTCATCCCTTGCA
PGP1.6mon.Batch2_X3_TTTGTCATCCTCTAGC
PGP1.6mon.Batch2_X3_TTTGTCATCGGCGCTA
PGP1.6mon.Batch2_X3_TTTGTCATCTCCAACC


In [20]:
to_concat = [an.read_h5ad(f'../data/SCP282/for_inference/{file}') for file in h5s]

In [21]:
combined_data = an.concat(to_concat)

In [147]:
combined_data.obs = combined_data.obs.reset_index(drop=True)

In [153]:
combined_data.obs = combined_data.obs.drop(['nGene', 'nUMI', 'Cluster'], axis=1)

In [156]:
for col in combined_data.obs.columns:
    combined_data.obs[col] = combined_data.obs[col].astype(str)

In [157]:
combined_data.write_h5ad('../data/SCP282/combined_data.h5ad')

In [128]:
data.var

,index
0,RP11-34P13.3
1,FAM138A
2,OR4F5
3,RP11-34P13.7
4,RP11-34P13.8
...,...
33689,AC233755.2
33690,AC233755.1
33691,AC240274.1
33692,AC213203.1


In [2]:
metadata = pd.read_csv('../data/SCP282/metadata/meta_combined.txt', sep='\t')

/var/folders/pd/jsjcl0fn7w57s5mfr34b20pm0000gn/T/ipykernel_51687/2208636785.py:1: DtypeWarning: Columns (1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv('../data/SCP282/metadata/meta_combined.txt', sep='\t')


In [6]:
combined_data.obs = combined_data.obs.drop('CellType', axis=1)

In [7]:
combined_data

,NAME,nGene,nUMI,Cluster,Organoid,CellType,Batch
0,TYPE,numeric,numeric,group,group,group,group
1,GM.6mon_X1_AAACCTGAGCAGGTCA,2253,5593,5,13,Cycling,GM8330_6mon
2,GM.6mon_X1_AAACCTGCAAGAGTCG,1137,2213,3,13,Immature Interneurons,GM8330_6mon
3,GM.6mon_X1_AAACCTGCAATGACCT,3084,9435,4,13,Cycling,GM8330_6mon
4,GM.6mon_X1_AAACCTGGTCAAAGAT,2437,7361,6,13,oRG/Astroglia,GM8330_6mon
...,...,...,...,...,...,...,...
166238,11a.6mon_X3_TTTGTCAGTACATCCA,1634,3951,15,12,IPCs,11a_6mon
166239,11a.6mon_X3_TTTGTCAGTAGGAGTC,2807,8397,11,12,oRG/Astroglia,11a_6mon
166240,11a.6mon_X3_TTTGTCAGTGTAACGG,1723,3694,5,12,Ventral Precursors,11a_6mon
166241,11a.6mon_X3_TTTGTCAGTTTGACAC,1851,4112,3,12,Immature CPNs,11a_6mon


In [13]:
combined_data.obs = combined_data.obs.merge(metadata, on='NAME', how='inner')

/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/anndata/_core/anndata.py:798: UserWarning: 
AnnData expects .obs.index to contain strings, but got values like:
    [0, 1, 2, 3, 4]

    Inferred to be: integer

  value_idx = self._prep_dim_index(value.index, attr)


In [15]:
combined_data.obs

,NAME,Organoid_x,CellType_x,Batch_x,nGene,nUMI,Cluster,Organoid_y,CellType_y,Batch_y
0,PGP1.6mon.Batch2_X1_AAACCTGAGATCTGAA,19,CPNs,PGP1_6mon_Batch3,2145,5260,8,19,CPNs,PGP1_6mon_Batch3
1,PGP1.6mon.Batch2_X1_AAACCTGAGCCCAATT,19,oRG/Astroglia,PGP1_6mon_Batch3,3282,12256,2,19,oRG/Astroglia,PGP1_6mon_Batch3
2,PGP1.6mon.Batch2_X1_AAACCTGAGGATGCGT,19,oRG/Astroglia,PGP1_6mon_Batch3,2103,7619,2,19,oRG/Astroglia,PGP1_6mon_Batch3
3,PGP1.6mon.Batch2_X1_AAACCTGCACTTCTGC,19,Immature Interneurons,PGP1_6mon_Batch3,1533,3769,1,19,Immature Interneurons,PGP1_6mon_Batch3
4,PGP1.6mon.Batch2_X1_AAACCTGGTATATCCG,19,Ventral Precursors,PGP1_6mon_Batch3,1230,2658,6,19,Ventral Precursors,PGP1_6mon_Batch3
...,...,...,...,...,...,...,...,...,...,...
133278,PGP1.3mon_X3_TTTGTCACAAGAGGCT,3,Immature CPNs,PGP1_3mon_Batch1,1543,3239,1,3,Immature CPNs,PGP1_3mon_Batch1
133279,PGP1.3mon_X3_TTTGTCAGTCCATGAT,3,Immature CPNs,PGP1_3mon_Batch1,1967,4781,1,3,Immature CPNs,PGP1_3mon_Batch1
133280,PGP1.3mon_X3_TTTGTCATCAGCAACT,3,IPCs/Immature PNs,PGP1_3mon_Batch1,1342,2941,7,3,IPCs/Immature PNs,PGP1_3mon_Batch1
133281,PGP1.3mon_X3_TTTGTCATCAGGCAAG,3,Immature CPNs,PGP1_3mon_Batch1,1194,2215,1,3,Immature CPNs,PGP1_3mon_Batch1


In [14]:
combined_data.obs['Organoid_x'] == combined_data.obs['Organoid_y']

0         True
1         True
2         True
3         True
4         True
          ... 
133278    True
133279    True
133280    True
133281    True
133282    True
Length: 133283, dtype: bool

### Now we'll look at the classes in the data from the Allen Brain Institute

In [23]:
allen = an.read_h5ad('../data/benchmark/human.h5ad')
combined_data = an.read_h5ad('../data/benchmark/combined_organoid_data_for_test.h5ad')
allen_labels = pd.read_csv('../data/benchmark/human_labels.csv')

/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [24]:
allen.X.max(), combined_data.X.max()

(409205.0, 13.454478)

In [25]:
import scanpy as sc 

sc.pp.normalize_total(combined_data)
sc.pp.normalize_total(allen)

In [26]:
combined_data.X.max(), allen.X.max()

(41.361042, 326391.78)

In [31]:
allen.write_h5ad('../data/benchmark/human_cpm.h5ad')

In [27]:
allen_labels['subclass_label'].unique()

array([nan, 'VIP', 'LAMP5', 'IT', 'PAX6', 'Oligodendrocyte', 'Astrocyte',
       'L5/6 IT Car3', 'L5/6 NP', 'SST', 'L6 CT', 'OPC', 'PVALB', 'L6b',
       'Microglia', 'L5 ET', 'Pericyte', 'Endothelial', 'L4 IT', 'VLMC'],
      dtype=object)

In [28]:
combined_data.obs['CellType'].unique()

['CPNs', 'oRG/Astroglia', 'Immature Interneurons', 'Ventral Precursors', 'oRG', ..., 'Immature CPNs', 'Astroglia', 'CFuPNs', 'Immature CFuPNs', 'IPCs/Immature PNs']
Length: 15
Categories (15, object): ['Astroglia', 'CFuPNs', 'CPNs', 'Cycling', ..., 'Unknown', 'Ventral Precursors', 'oRG', 'oRG/Astroglia']

In [29]:
mapping = {
    'CPNs' : ['IT', 'L5/6 IT Car3', 'L4 IT'],
    'oRG/Astroglia': ['PAX6', 'Astrocyte'],
    'Immature Interneurons': ['SST', 'VIP'],
    'Ventral Precursors': ['VIP', 'LAMP5', 'Astrocyte', 'SST', 'OPC'], 
    'oRG': ['SOX6', 'Astrocyte', 'L5/6 NP'], 
    'Unknown': ['SOX6', 'Astrocyte'], # Anything 
    'Cycling': ['SOX6'], 
    'RG': ['SOX6', 'Astrocyte'],
    'Immature PNs': ['IT', 'PAX6', 'L5/6 IT Car3', 'L5/6 NP', 'L6 CT', 'L6b', 'L5 ET', 'L4 IT'], 
    'IPCs': ['SOX6', 'Astrocyte', 'OPC'], 
    'Immature CPNs': ['IT', 'L5/6 IT Car3', 'L4 IT'],
    'Astroglia': ['PAX6', 'Astrocyte', 'Oligodendrocyte'], 
    'CFuPNs': ['L5/6 NP', 'L6b', 'L5 ET', 'L6 CT'],
    'Immature CFuPNs': ['L5/6 NP', 'L6b', 'L5 ET', 'L6 CT'], 
    'IPCs/Immature PNs': ['IT', 'PAX6', 'Oligodendrocyte', 'Astrocyte', 'L5/6 IT Car3', 'L5/6 NP', 'L6 CT', 'L6b', 'L5 ET', 'L4 IT']
}

There should be no microglia in the organoid data

### Training our model for inference on the organoid data

While we figure out docker, we can train a model locally and use it for inference.

In [32]:
combined_genes = [x.upper() for x in combined_data.var['index'].values]
currgenes = [x.upper() for x in allen.var.index]
refgenes = list(set(currgenes).intersection(combined_genes))
    
module = DataModule(
    datafiles=['../data/benchmark/human_cpm.h5ad'],
    labelfiles=['../data/benchmark/human_labels_clean.csv'],
    class_label='subclass_label',
    index_col='cell',
    batch_size=16,
    num_workers=0,
    deterministic=True,
    normalize=True,
    currgenes=currgenes,
    refgenes=refgenes,
)

module.prepare_data()
module.setup()

wandb_logger = WandbLogger(
    project=f"Human model for organoid benchmark",
)

lr_callback = pl.callbacks.LearningRateMonitor(logging_interval='epoch')

upload_callback = UploadCallback(
    path='checkpoints',
    desc=f'local_allen_model_for_organoid_inference'
)

early_stopping_callback = pl.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=20,
)

trainer = pl.Trainer(
    gpus=(1 if torch.cuda.is_available() else 0),
    auto_lr_find=False,
    logger=wandb_logger,
    max_epochs=500,
    gradient_clip_val=0.5,
    callbacks=[
        lr_callback, 
        upload_callback,
        early_stopping_callback,
    ],
    stochastic_weight_avg=True,
    
)


model = SIMSClassifier(
    input_dim=module.num_features,
    output_dim=module.num_labels,
    weights=module.weights,
)

print(f'Input dim and output dim are {module.num_features} / {module.num_labels}')

trainer.fit(model, datamodule=module)

Creating train/val/test DataLoaders...
Done, continuing to training.
Calculating weights


wandb: Currently logged in as: jlehrer1. Use `wandb login --relogin` to force relogin


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Initializing network
Initializing explain matrix
Input dim and output dim are 21730 / 19
Creating train/val/test DataLoaders...
Done, continuing to training.
Calculating weights



  | Name    | Type   | Params
-----------------------------------
0 | network | TabNet | 1.4 M 
-----------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
5.760     Total estimated model params size (MB)


...

Validation: 0it [00:00, ?it/s]

Saving checkpoint at epoch 8


Validation: 0it [00:00, ?it/s]

Saving checkpoint at epoch 9


/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:724: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


Now let's set up the data to pass into the classifier for the predictions

In [59]:
from scsims.data import CollateLoader 
from scsims.testing import TestAnndatasetMatrix

traindataset = TestAnndatasetMatrix(
    combined_data.X,
)

testloader = CollateLoader(
    dataset=traindataset, 
    batch_size=4, 
    num_workers=0, 
    refgenes=refgenes, 
    currgenes=combined_genes,
    normalize=True,
)

In [34]:
from functools import partial
tqdm = partial(tqdm, position=0, leave=True)

In [70]:
model = SIMSClassifier.load_from_checkpoint(
    'checkpoints/checkpoint-9-desc-local_allen_model_for_organoid_inference.ckpt',
    input_dim=module.num_features,
    output_dim=module.num_labels,
)

Initializing network
Initializing explain matrix


In [60]:
next(iter(testloader))

tensor([[0.0227, 0.0000, 0.0000,  ..., 0.0200, 0.0000, 0.0000],
        [0.0179, 0.0000, 0.0000,  ..., 0.0155, 0.0000, 0.0155],
        [0.0000, 0.0000, 0.0000,  ..., 0.0197, 0.0000, 0.0000],
        [0.0240, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

In [71]:
preds = []

model.eval()
with torch.no_grad():
    for X in tqdm(testloader):
        res, _ = model(X)
        
        _, top_preds = res.topk(3, axis=1) # to get indices
        preds.extend(top_preds.numpy())

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33321/33321 [15:16:33<00:00,  1.65s/it]


In [72]:
preds = pd.DataFrame(preds)

In [73]:
len(preds)

133283

In [74]:
combined_data.obs['predicted'] = preds.loc[:, 0].values

In [75]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le = le.fit([x for x in allen_labels['subclass_label'].unique()])

In [76]:
combined_data.obs['predicted_str'] = le.inverse_transform(combined_data.obs['predicted'])

In [77]:
acc = 0
for truth, pred in zip(combined_data.obs["CellType"], combined_data.obs["predicted_str"]):
    options = mapping[truth]
    if pred in options:
        acc += 1

In [78]:
acc / len(combined_data.obs)

0.13609387543797782

In [ ]:
allen